In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install simpletransformers


In [3]:
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel,Seq2SeqArgs

In [4]:
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 2
model_args.no_save = True
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Initialize model
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
    use_cuda=False,
    fp16 = False,
    
)

In [5]:
train_data = pd.read_csv('../input/serb-summarizer/train.csv')

In [6]:
train_data.head()

In [7]:
train_data.drop('FileName', axis = 1, inplace= True)
train_data.rename(columns = {'Abstract':'input_text', 'RHS':'target_text'}, inplace = True)
train_data

In [8]:
train_df = pd.DataFrame(
    train_data, columns=["input_text", "target_text"]
)

In [9]:
train_df

In [10]:
eval_data = pd.read_csv('../input/serb-summarizer/val.csv')

In [11]:
eval_data.drop('FileName', axis = 1, inplace= True)
eval_data.rename(columns = {'Abstract':'input_text', 'RHS':'target_text'}, inplace = True)
eval_data

In [12]:
eval_df = pd.DataFrame(
    eval_data, columns=["input_text", "target_text"]
)

In [13]:
train_df

In [14]:
eval_df

In [15]:
model.train_model(train_df, eval_data=eval_df)

In [21]:
results = model.eval_model(eval_df)

In [22]:
results

In [24]:
model


In [26]:
from joblib import Parallel, delayed
import joblib
  
  
# Save the model as a pickle in a file
joblib.dump(model, 'my_model.pkl')

In [63]:
print(model.predict(
        [
            "Interviews were conducted with thirteen professional designers to understand their attitudes towards fixation and the practices they adopt to address it . Fixation was thought to be encouraged and discouraged by a wide range of factors related to the project the client the design team the organisational culture and the design activities employed . The experiences that designers accumulate during their professional lives were associated with fixation in different ways . The experience of prior design failures was thought to encourage fixation whilst the experience of varied solutions was thought to discourage fixation . Recognising fixation episodes and reflecting on them was described as the means by which designers could guard against such episodes in the future and thus be more creative . "        ]
    )
)

In [64]:
ref_summary = ["Interviews were conducted with thirteen professional designers to understand their attitudes towards fixation and the practices they adopt to address it . Fixation was thought to be encouraged and discouraged by a wide range of factors related to the project the client the design team the organisational culture and the design activities employed . The experiences that designers accumulate during their professional lives were associated with fixation in different ways . The experience of prior design failures was thought to encourage fixation whilst the experience of varied solutions was thought to discourage fixation . Recognising fixation episodes and reflecting on them was described as the means by which designers could guard against such episodes in the future and thus be more creative."]
candidate_summary = ["We surveyed thirteen professional designers on their attitudes towards fixation and their practices to address"]


df_test = pd.DataFrame()
df_test['Pred'] = pd.DataFrame(candidate_summary)
df_test

In [65]:
!pip install rouge_score

In [67]:
from datasets import load_metric
import re

metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result


for i in range(10):
    candidate_summaries = list(df_test['Pred'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i + 1])))
    print(f"First {i + 1} sentences: Scores {calc_rouge_scores(candidate_summaries, ref_summary)}")

In [72]:
pred = []
for i in range(100):
    text = eval_data["input_text"][i]
    prediction = model.predict([text])
    pred.append(prediction)

In [74]:
# print(pred)
df_pred = pd.DataFrame(pred)
df_pred

In [96]:
#prediction on test data
print(model.predict(
        [
             "We explore the notion of a Well spaced Blue noise Distribution WBD of points which combines two desirable properties. First the point distribution is random as measured by its spectrum having blue noise. Second it is well spaced in the sense that the minimum separation distance between samples is large compared to the maximum coverage distance between a domain point and a sample i.e. its Voronoi cell aspect ratios are small. It is well known that maximizing one of these properties destroys the other uniform random points have no aspect ratio bound and the vertices of an equilateral triangular tiling have no randomness. However we show that there is a lot of room in the middle to get good values for both. Maximal Poisson disk sampling provides and blue noise. We show that a standard optimization technique can improve the well spacedness while preserving randomness. Given a random point set our Opt beta algorithm iterates over the points and for each point locally optimizes its Voronoi cell aspect ratio . It can improve to a large fraction of the theoretical bound given by a structured tiling improving from 1.0 to around 0.8 about half way to 0.58 while preserving most of the randomness of the original set. In terms of both and randomness the output of Opt compares favorably to alternative point improvement techniques such as centroidal Voronoi tessellation with a constant density function which do not target directly. We demonstrate the usefulness of our output through meshing and filtering applications. An open problem is constructing from scratch a WBD distribution with a guarantee of beta 1."        ]
    )
)

In [93]:
df_pred.rename(columns={0: "summary"}, inplace = True)
df_pred
df_val = pd.read_csv('../input/serb-summarizer/val.csv')
df_ref = df_val['RHS']
ref_summary = list(df_ref)
print(ref_summary)

In [95]:
from datasets import load_metric
import re

metric = load_metric('rouge')

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result


for i in range(10):
    candidate_summaries = list(df_pred['summary'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i + 1])))
    print(f"First {i + 1} sentences: Scores {calc_rouge_scores(candidate_summaries, ref_summary)}")